# Background

This experiment is based on the findings in

All Bark and No Bite: Rogue Dimensions in Transformer Language Models Obscure Representational Quality

by William Timkey and Marten van Schijndel

https://arxiv.org/pdf/2109.04404.pdf

who find that a few dimensions in BERT embeddings are responsible for much of the variation in cosine similarity score. Furthermore, those dimensions are not necessarily important to the behavior of the model.

The authors investigate a few post-processing techniques to temper the influence of these 'rogue dimensions', and find z-scoring of embeddings to be the most accurate.

In this notebook I take the multi-prototype embeddings used in our experiments and apply the z-scoring technique to them. We hypothesize that the normalized embeddings will be better inputs for label propagation.

Of the methods we tested for property inference at the token level, label propagation did the worst. There is reason to believe that label propagation, but not the ffnn or plsr, is affected by rogue dimensions. This is because label propagation provides a raw vector as output, whereas PLSR and FFNN are allowed to change the weights of the dimensions. This, for label propagation, the final feature space is the same as BERT space rather than a transformation of it. Therefore, it likely also has this property of rogue dimensions. If removing rogue dimensions through post-processing improve performance on label propagation, then we validate this hypothesis. 

# Step 1

read in multiprototype embeddings

In [1]:
import sys
sys.path.append("../src/")
sys.path.append("..")

import pandas as pd
import torch
from bert import *
from feature_data import *
from multiprototype import *
# import classifier_main
# import csv
# from nltk.corpus.reader.wordnet import Lemma
# from nltk.corpus import wordnet as wn
# from nltk.stem import WordNetLemmatizer

# import inflect
# import os
# from scipy.stats import spearmanr, pearsonr
# import re
# from src.utils import *


In [19]:
embedding_file = '../data/processed/multipro_embeddings/layer'+ '8' + 'clusters' + '5' + '.txt'
embs = read_multiprototype_embeddings(embedding_file, layer=8, num_clusters=5)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
Read in 12348 vectors of size 5 X 768


In [5]:
embs.get_embedding('dog')

array([[ 0.96497798,  0.25925376,  0.43838646, ..., -0.56938209,
        -0.16793092,  0.70900637],
       [ 0.92773051,  0.00529362,  0.45928537, ..., -0.28655665,
        -0.10632126,  0.8034243 ],
       [ 0.73461566,  0.19206844,  0.4664031 , ..., -0.35237304,
         0.14169224,  0.29012396],
       [ 1.00830649,  0.49483381,  0.4636305 , ..., -0.4081229 ,
         0.07974593,  0.48738606],
       [ 0.94780814,  0.3969447 ,  0.13683058, ..., -0.54753831,
         0.12271835,  0.53106732]])

# Step 2

transform by z-score

### calculate the mean vector of the whole lexicon

Concretely, given some corpus of length |M| containing word representations
x ∈ Rd , we compute the mean vector μ ∈ Rd

μ = 1 / |M| · ∑   x
              x∈M
              

In [5]:
M = embs.vocab_size * embs.num_prototypes
M

61740

In [13]:
embs.vectors.shape

(12348, 5, 768)

In [14]:
mu = np.average(embs.vectors, axis=0)
mu = np.average(mu, axis=0)

In [15]:
mu.shape

(768,)

### calculate the standard deviation in each dimension σ ∈ Rd


σ = √( 1 / |M| · ∑ (x − μ)^2 )
                x∈O


In [6]:
reshape = embs.vectors.copy().reshape(61740, 768)
reshape.shape

(61740, 768)

In [7]:
#### or we can just use stats to calculate zscore automatically

import scipy.stats as stats

zscored = stats.zscore(reshape, axis=0)



In [8]:
zscored.shape

(61740, 768)

In [9]:
rereshape = zscored.reshape(embs.vectors.shape[0], 5, 768)
rereshape.shape

(12348, 5, 768)

In [ ]:
# look to see that we've made changes, and they are proportional

In [10]:
rereshape[0]

array([[ 0.12651942,  0.53887696,  0.76818603, ...,  0.39137034,
         0.60938799, -0.83323767],
       [-0.71688614,  0.27605432,  0.53013951, ..., -0.22159489,
         0.74296869, -0.54277801],
       [-0.40535745,  1.21878709,  0.32580311, ...,  0.07791547,
         1.14954659, -0.60574313],
       [ 0.79435884,  0.50104107,  0.53370407, ..., -1.53231032,
         0.34156427, -0.80184054],
       [ 0.46793818,  0.77903953,  0.51473654, ..., -0.76861262,
         0.73271411, -1.00538076]])

In [11]:
embs.vectors[0]

array([[ 0.29584812,  0.16930976,  0.54239129, ..., -0.07648242,
         0.27238423, -0.77687997],
       [-0.21397193,  0.0323872 ,  0.44932853, ..., -0.29248422,
         0.32353501, -0.60838436],
       [-0.02565971,  0.52352216,  0.36944453, ..., -0.18694026,
         0.47922203, -0.64491042],
       [ 0.69954231,  0.14959842,  0.45072207, ..., -0.75436504,
         0.16982903, -0.7586665 ],
       [ 0.50222822,  0.29442712,  0.44330684, ..., -0.48524688,
         0.31960833, -0.87674016]])

In [12]:
embs.vectors.shape

(12348, 5, 768)

# Step 3

write to disk as normalized embeddings



In [13]:
num_prototypes = 5
outfile = '../data/processed/multipro_embeddings/layer8clusters5zscored.txt'
o = open(outfile, 'w')

for i in range(0,len(rereshape)):
    word = embs.word_indexer.get_object(i)
    mpro_emb = rereshape[i]
    for j in range(0, num_prototypes-1):
        vec = mpro_emb[j]
        number_str = np.array2string(vec, precision=8, max_line_width=None)
        number_str = number_str[1:-1].replace("\n", "") # get rid of brackets
        emb_str = word + ' ' + number_str + "\n"
        o.write(emb_str)

# Step 4

compare cosine distances with normal and non normalized embeddings, to test the post processing



In [25]:
normalized_embedding_file = '../data/processed/multipro_embeddings/layer'+ '8' + 'clusters' + '5' + 'zscored.txt'
#normalized = read_multiprototype_embeddings(normalized_embedding_file, layer=8, num_clusters=5)


from scipy.spatial.distance import cosine


cat = embs.get_embedding('cat')[0]
dog = embs.get_embedding('dog')[0]

cosine(cat, dog)




0.22859758363343563

In [35]:
normalized = embs
normalized.vectors = rereshape


In [36]:
cat

array([ 2.57397075e-01,  5.50991574e-02,  5.09894003e-01,  2.91784481e-01,
        2.34583793e-01,  1.83844776e-01,  4.06035144e-01,  5.79005592e-01,
        3.78582017e-01,  2.09627186e-01, -2.14448448e-01, -4.39438906e-01,
       -2.96734031e-01, -5.47690885e-01, -7.69823180e-01, -1.90098335e-01,
        3.09407421e-01,  3.53970601e-01, -9.79997115e-01,  6.11743967e-01,
       -1.54826929e-03, -3.06745571e-01, -7.96704255e-01, -6.91591620e-02,
        3.92397963e-01,  1.58124346e-01,  3.54863269e-01,  4.24258640e-01,
       -7.30701840e-02, -7.04413447e-01, -2.44069869e-01, -3.70271493e-01,
        5.64302971e-01,  5.47382167e-01, -6.43694423e-01,  6.96038241e-01,
       -9.96761102e-01, -4.04378109e-03, -1.82002020e-01, -2.39454369e-01,
        1.27447385e+00,  3.10663586e-01, -4.04041240e-01,  6.12389090e-01,
        9.33941835e-01,  4.30453276e-01, -3.62594197e-01, -1.39632770e-01,
        1.04750238e+00,  5.04864473e-01, -9.68065628e-03,  5.07830856e-01,
       -1.07120983e+00,  

In [37]:
cat = normalized.get_embedding('cat')[0]
dog = normalized.get_embedding('dog')[0]

cosine(cat, dog)

0.31459198030744495

In [38]:
cat

array([ 6.29090869e-02,  3.19649884e-01,  6.85060764e-01,  6.04855075e-01,
       -2.36949609e-01,  2.83565114e-01,  8.99859437e-01,  3.68190991e-01,
        9.52795018e-01,  4.75247193e-01, -6.73828063e-01, -1.47317091e+00,
       -7.31016479e-01, -1.78855884e+00, -8.65788447e-01, -1.42525702e+00,
        4.14151512e-01,  5.95467821e-01, -1.93789946e+00,  9.45461274e-01,
       -4.45878529e-01, -1.07889553e+00, -1.19801332e+00, -6.26564718e-01,
        5.32159879e-01,  6.58965574e-02,  9.79277174e-01,  8.97568180e-01,
        3.89153451e-01, -1.32334133e+00, -1.06198118e+00, -1.23594588e+00,
        8.00685757e-01,  7.93932102e-01, -8.37384005e-01,  1.73964854e+00,
       -6.15140923e-01, -3.46223600e-02, -9.50693403e-01, -2.68572989e-01,
        2.68104934e+00,  1.10479761e+00,  8.44521236e-02,  5.14806550e-01,
        1.71540586e+00,  1.32772163e+00, -7.75997405e-01, -4.06874920e-02,
        1.59414388e+00,  1.10335459e+00,  8.92811799e-01,  8.65188090e-01,
       -1.31674637e+00,  

In [39]:
normalized.vectors[0]

array([[ 0.12651942,  0.53887696,  0.76818603, ...,  0.39137034,
         0.60938799, -0.83323767],
       [-0.71688614,  0.27605432,  0.53013951, ..., -0.22159489,
         0.74296869, -0.54277801],
       [-0.40535745,  1.21878709,  0.32580311, ...,  0.07791547,
         1.14954659, -0.60574313],
       [ 0.79435884,  0.50104107,  0.53370407, ..., -1.53231032,
         0.34156427, -0.80184054],
       [ 0.46793818,  0.77903953,  0.51473654, ..., -0.76861262,
         0.73271411, -1.00538076]])

In [40]:
normalized.get_embedding('abandon')

array([[ 0.12651942,  0.53887696,  0.76818603, ...,  0.39137034,
         0.60938799, -0.83323767],
       [-0.71688614,  0.27605432,  0.53013951, ..., -0.22159489,
         0.74296869, -0.54277801],
       [-0.40535745,  1.21878709,  0.32580311, ...,  0.07791547,
         1.14954659, -0.60574313],
       [ 0.79435884,  0.50104107,  0.53370407, ..., -1.53231032,
         0.34156427, -0.80184054],
       [ 0.46793818,  0.77903953,  0.51473654, ..., -0.76861262,
         0.73271411, -1.00538076]])

In [42]:
embs.get_embedding('abandon')

array([[ 0.29584812,  0.16930976,  0.54239129, ..., -0.07648242,
         0.27238423, -0.77687997],
       [-0.21397193,  0.0323872 ,  0.44932853, ..., -0.29248422,
         0.32353501, -0.60838436],
       [-0.02565971,  0.52352216,  0.36944453, ..., -0.18694026,
         0.47922203, -0.64491042],
       [ 0.69954231,  0.14959842,  0.45072207, ..., -0.75436504,
         0.16982903, -0.7586665 ],
       [ 0.50222822,  0.29442712,  0.44330684, ..., -0.48524688,
         0.31960833, -0.87674016]])